In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os

from msfm.utils import cosmogrid, prior
from msfm.utils.input_output import read_yaml

### load constants

In [3]:
conf_dir = "../../configs/config.yaml"
conf = read_yaml(conf_dir)

meta_info_file = os.path.join("../..", conf["files"]["meta_info"])
cosmo_params_info = cosmogrid.get_cosmo_params_info(meta_info_file, "grid")

n_cosmos = conf["analysis"]["grid"]["n_cosmos"]

### collect all of the cosmologies/parameters

In [4]:
cosmos = []
for i in range(n_cosmos):
    cosmo = [cosmo_params_info[cosmo_param][i] for cosmo_param in conf["analysis"]["params"][:6]]
    cosmo = np.array(cosmo, dtype=np.float32)

    cosmos.append(cosmo)

cosmos = np.stack(cosmos)
print(cosmos.shape)

(2500, 6)


### test whether the full CosmoGrid is within the prior
Which it should be

In [11]:
bad_cosmos = np.arange(n_cosmos)[~np.squeeze(prior.in_grid_prior(cosmos))]
print(f"sobol index of cosmologies not included in the prior {bad_cosmos}")

for i in bad_cosmos:
    ["Om", "s8", "H0", "Ob", "ns", "w0"]
    cosmo = [cosmo_params_info[cosmo_param][i] for cosmo_param in conf["analysis"]["params"][:6]]
    cosmo = np.array(cosmo, dtype=np.float32)

    sobol = cosmo_params_info["sobol_index"][i]

    # print(sobol)
    # print(cosmo)
    # print("\n")

print(f"number of not included cosmologies {len(bad_cosmos)}")

23-02-22 10:35:29  analysis.py INF   Loaded the config 
23-02-22 10:35:29  analysis.py INF   Loaded the config 
sobol index of cosmologies not included in the prior [  7  69  73  75  76  78  92 102 140 353 523 533]
number of not included cosmologies 12


### original KiDS

In [12]:
from kids1000_analysis.probability import kids1000_prior

In [13]:
kids_cosmos = []
for i in range(n_cosmos):
    kids_cosmo = [cosmo_params_info[cosmo_param][i] for cosmo_param in ["Om", "s8", "H0", "Ob", "ns", "w0"]]
    kids_cosmo = np.array(kids_cosmo, dtype=np.float32)
    # H0 to h conversion
    kids_cosmo[2] = kids_cosmo[2]/100

    kids_cosmos.append(kids_cosmo)

kids_cosmos = np.stack(kids_cosmos)
print(kids_cosmos.shape)

(2500, 6)


In [14]:
np.sum(kids1000_prior(kids_cosmos))

2488